In [1]:
%load_ext autoreload
%autoreload 2
from get_data import *


In [3]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from catboost import CatBoostRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
import pandas as pd

PATH_TRAIN = "data/train_15_2021-09-01_2022-02-28"

y = get_data(
    "20"
)
y_train = y[:-3]
y_test = y[len(y_train):]

forecaster = ForecasterAutoreg(
    regressor = CatBoostRegressor(random_state=123, silent=True),
    lags = 12
)

forecaster.fit(y=y_train)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_15_2021-09-01_2022-02-28'

In [4]:
y_pred = forecaster.predict(steps=3)
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(list(y_test), list(y_pred)))
print(list(y_test))
print(list(y_pred))


217.6984431556654
[37784.99, 38299.39, 38040.27]
[38224.79914825784, 38091.15910516387, 38035.21471362698]


In [5]:
%load_ext autoreload
%autoreload 2
from get_data import *
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from catboost import CatBoostRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

df_test = get_data(
    "2022-01-01",
    "2022-04-05",
    "BTCUSDT",
    "15m"
)
y_test = df_test["high"]

forecaster_catboost = ForecasterAutoreg(
    regressor=CatBoostRegressor(random_state=123, silent=True),
    lags = 20
)

metric_catboost, predictions = backtesting_forecaster(
    forecaster=forecaster_catboost,
    y=y_test,
    initial_train_size=200,
    fixed_train_size=True,
    steps=1,
    refit=True,
    metric='mean_squared_error',
    verbose=False # Change to True to see detailed information
)

print(f"Catboost res: {metric_catboost}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Forecaster will be fit 8824 times. This can take substantial amounts of time. If not feasible, try with `refit = False`. 



KeyboardInterrupt: 

In [4]:
forecaster_xgboost = ForecasterAutoreg(
    regressor=XGBRegressor(random_state=123),
    lags = 12
)

metric_xgboost, predictions = backtesting_forecaster(
    forecaster=forecaster_xgboost,
    y=y_test,
    initial_train_size=1000,
    fixed_train_size=True,
    steps=3,
    refit=True,
    metric='mean_absolute_error',
    verbose=False # Change to True to see detailed information
)

print(f"XGBoost res: {metric_xgboost}")

Forecaster will be fit 595 times. This can take substantial amounts of time. If not feasible, try with `refit = False`. 

XGBoost res: 145.0879571953781


In [5]:
forecaster_lightgbm = ForecasterAutoreg(
    regressor=LGBMRegressor(random_state=123),
    lags = 12
)

metric_lightgbm, predictions = backtesting_forecaster(
    forecaster=forecaster_lightgbm,
    y=y_test,
    initial_train_size=1000,
    fixed_train_size=True,
    steps=3,
    refit=True,
    metric='mean_absolute_error',
    verbose=False # Change to True to see detailed information
)

print(f"LightGBM res: {metric_catboost}")

Forecaster will be fit 595 times. This can take substantial amounts of time. If not feasible, try with `refit = False`. 

LightGBM res: 138.59208257376315


In [6]:
print(metric_lightgbm)

159.0022985628589


In [12]:
print(len(predictions))

16270


In [5]:
print(metric)

138.59208257376315


In [17]:
%load_ext autoreload
%autoreload 2
from get_data import *
from sklearn.metrics import mean_absolute_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from catboost import CatBoostRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
import pandas as pd

def test_regressor(data, regressor, window, lags, steps_forward):
    error = 0.0
    iters = 0
    for i in tqdm(range(0, len(data) - window)):
        left = i
        right = left + window
        tmp = data[left : right]
        y_train = tmp[:-steps_forward]
        y_test_true = tmp[len(y_train):]
        forecaster = ForecasterAutoreg(
            regressor=regressor,
            lags=lags
        )
        forecaster.fit(y=y_train)
        y_pred = forecaster.predict(steps=steps_forward)
        error += mean_absolute_error(list(y_test_true), list(y_pred))
        iters += 1
    
    return error / iters
        
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
PATH_TRAIN = "data/test_15_2022-03-01_2022-03-30"
y = pd.Series(get_high_from_data(PATH_TRAIN))

test_regressor(
    data=y,
    regressor=CatBoostRegressor(random_state=123, silent=True),
    window=1000,
    lags=12,
    steps_forward=3
)

100%|██████████| 1785/1785 [32:37<00:00,  1.10s/it]


136.9468653416249

In [6]:
PATH_TRAIN = "data/val_15_2022-03-20_2022-03-30"
y = pd.Series(get_high_from_data(PATH_TRAIN))

test_regressor(
    data=y,
    regressor=CatBoostRegressor(random_state=123, silent=True),
    window=500,
    lags=12,
    steps_forward=3
)

100%|██████████| 461/461 [08:03<00:00,  1.05s/it]


129.30586903628384

In [14]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

test_regressor(
    data=y,
    regressor=XGBRegressor(random_state=123),
    window=500,
    lags=12,
    steps_forward=3
)

100%|██████████| 461/461 [00:54<00:00,  8.47it/s]


133.67331649945763

In [15]:
test_regressor(
    data=y,
    regressor=LGBMRegressor(random_state=123),
    window=500,
    lags=12,
    steps_forward=3
)

100%|██████████| 461/461 [00:21<00:00, 21.04it/s]


173.1785846478238

### TESTING FOR LOWER ###


In [21]:
PATH_TRAIN = "data/val_15_2022-03-20_2022-03-30"
y = pd.Series(get_low_from_data(PATH_TRAIN))

test_regressor(
    data=y,
    regressor=CatBoostRegressor(random_state=123, silent=True),
    window=500,
    lags=12,
    steps_forward=3
)

100%|██████████| 461/461 [07:26<00:00,  1.03it/s]


128.84500271688165

In [19]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

test_regressor(
    data=y,
    regressor=XGBRegressor(random_state=123),
    window=500,
    lags=12,
    steps_forward=3
)

100%|██████████| 461/461 [00:57<00:00,  8.06it/s]


141.0583876762473

In [20]:
test_regressor(
    data=y,
    regressor=LGBMRegressor(random_state=123),
    window=500,
    lags=12,
    steps_forward=3
)

100%|██████████| 461/461 [00:16<00:00, 28.20it/s]


170.04839754968978